In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
df=pd.read_csv('/kaggle/input/moneyball-mlb-stats-19622012/baseball.csv')
df.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.subplots(figsize=(15,10))
sns.heatmap(df.corr(),annot=True)

In [ ]:
df.corr()

In [ ]:
df_playoff=df.groupby('Playoffs')
df_playoff_0=df_playoff.get_group(0)
df_playoff_1=df_playoff.get_group(1)
# Plotting scatterplotplt.
#figure(figsize=(10,8))
plt.scatter(x=df_playoff_0.W,y = df_playoff_0.RS,c="red")
plt.scatter(x=df_playoff_1.W,y = df_playoff_1.RS,c="blue")
plt.xlabel("Wins")
plt.ylabel("Runs Scored")
plt.axvline(x = 100)
plt.axhline(y=800)
## To enter into the playoff minimum of 100 wins is required.

In [ ]:
df_playoff=df.groupby('Playoffs')
df_playoff_0=df_playoff.get_group(0)
df_playoff_1=df_playoff.get_group(1)
# Plotting scatterplotplt.
#figure(figsize=(10,8))
plt.scatter(x=df_playoff_0.W,y = df_playoff_0.RA,c="red")
plt.scatter(x=df_playoff_1.W,y = df_playoff_1.RA,c="blue")
plt.xlabel("Wins")
plt.ylabel("Runs Scored")
plt.axvline(x=100)
plt.axhline(y=600)
## To get a clear win runs allowed should be less than 600.

In [ ]:
#RA - Runs Allowed
#RS - Runs Scored
#OBP - On Base Percentage
#SLG - Slugging Percentage
#BA - Batting Average
#OOBP - Opponent's OBP
#OSLG - Opponent's SLG
#W - No of wins in that season
#Statistical Analysis Clip from Moneyball (2011)

In [ ]:
## Some columnns like Playoffs, RankSeason, RankPlayoffs information is available only after the game is over.
## So dropping these columns.
df.drop(['Playoffs', 'RankSeason', 'RankPlayoffs'],axis=1,inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
df.shape

In [ ]:
## From the correlation matrix
# The RS and RA are having inverse correaltion i.e if more runs allowed the chances of winning will be less.
# Similarly OBP, SLG are highly correlated with W (wins).
# Simarly OOBP,OSLG are negatively correlated with W, which states more and more run you allowed the chances of winning reduces.
# RS is highly correlated with OBP,SLG.
# RA is highly correlated with OOBP, OSLG.

In [ ]:
df['Year'].unique()

In [ ]:
## We are replicating the year 2002, so only considering the year before 2002 to predict the win in 2002.
## Replicate data before 2002
df1=df[df['Year']<2002].reset_index(drop=True)

In [ ]:
df1['RD']=df1['RS']-df1['RA']
df1.head()

In [ ]:
df_med=df1[['RS','RA','OBP','SLG','OOBP','OSLG','W']].reset_index(drop=True)

In [ ]:
## Lets check the ols summary.
import statsmodels.api as sm

df_med['OOBP'].fillna(value=df_med['OOBP'].median(),inplace=True)
df_med['OSLG'].fillna(value=df_med['OSLG'].median(),inplace=True)
X=df_med[['RS','RA','OBP','SLG','OOBP','OSLG']]
xc=sm.add_constant(X)
Y=df_med['W']
model=sm.OLS(Y,xc).fit()
model.summary()

In [ ]:
## Only RA, RS model pass the statistical test.
## But due to high correlation between the dependent columns, it is better to work on different models than on one single model.
## So one model will be RA=beta0+beta1*OOBP+beta2*OSLG
## Second model will be RS=beta0+beta1*OBP+beta2*SLG.
## and the wins will be predicted with the RD (Runs Difference)
## The equation will be W=beta0+beta1*RD
## Please note the beta values are different for every model.

In [ ]:
## Again looking for the correlation matrix
import matplotlib.pyplot as plt
import seaborn as sns
plt.subplots(figsize=(15,10))
sns.heatmap(df_med.corr(),annot=True)

In [ ]:
## The new columns RD is highly correlated with the W column.
# Now dropping all the other irrelevant columns.
df1.drop(['Team','League','Year'],axis=1,inplace=True)

In [ ]:
df1.head()

In [ ]:
## Going for the linear test, so checking distribution of data.
for i in df1.columns:
    df1.hist(i)

In [ ]:
Here are the Oakland Athletics statistics in 2001 before the playoffs.

The production data is already given,therefore aim is not to bulid a model but to check how model perform.

OBP: 0.339

SLG: 0.430

OOBP: 0.307

OSLG: 0.373 Lets plug in these values into the above models to generate predictions.

In [ ]:
## Median Imputation model building.

In [ ]:
df_med.shape

In [ ]:
# Extracting our variables from the dataframe.
from sklearn.linear_model import LinearRegression
x = df_med[['OBP','SLG']].values
y = df_med[['RS']].values# Calling our model object.
RS_model = LinearRegression()# Fitting the model.
RS_model.fit(x,y)# Printing model intercept and coefficients.
print(RS_model.intercept_)
print(RS_model.coef_)

In [ ]:
# Extracting our variables from the dataframe.
x = df_med[['OOBP','OSLG']].values
y = df_med[['RA']].values# Calling our model object.
RA_model = LinearRegression()# Fitting the model.
RA_model.fit(x,y)# Printing model intercept and coefficients.
print(RA_model.intercept_)
print(RA_model.coef_)

In [ ]:
df_med['RD']=df1['RD']
x = df_med[['RD']].values
y = df_med[['W']].values# Calling our model object.
W_model = LinearRegression()# Fitting the model.
W_model.fit(x,y)# Printing model intercept and coefficients.
print(W_model.intercept_)
print(W_model.coef_)

In [ ]:
RS_model.predict([[0.339,0.430]])

In [ ]:
RA_model.predict([[0.307,0.373]])

In [ ]:
W_model.predict(RS_model.predict([[0.339,0.430]])-RA_model.predict([[0.307,0.373]]))

In [ ]:
## Dropping any null values
#df_drop=df1.dropna()
df_drop.shape

In [ ]:
df_drop=df1.dropna()
from sklearn.linear_model import LinearRegression
# Extracting our variables from the dataframe.
x = df_drop[['OBP','SLG']].values
y = df_drop[['RS']].values# Calling our model object.
RS_model = LinearRegression()# Fitting the model.
RS_model.fit(x,y)
# Printing model intercept and coefficients.
print(RS_model.intercept_)
print(RS_model.coef_)

In [ ]:
# Extracting our variables from the dataframe.
x = df_drop[['OOBP','OSLG']].values
y = df_drop[['RA']].values# Calling our model object.
RA_model = LinearRegression()# Fitting the model.
RA_model.fit(x,y)# Printing model intercept and coefficients.
print(RA_model.intercept_)
print(RA_model.coef_)

In [ ]:

x = df_drop[['RD']].values
y = df_drop[['W']].values# Calling our model object.
W_model = LinearRegression()# Fitting the model.
W_model.fit(x,y)# Printing model intercept and coefficients.
print(W_model.intercept_)
print(W_model.coef_)


In [ ]:
# Prediction for runs scored.
RS_model.predict([[0.339,0.430]])


In [ ]:
RA_model.predict([[0.307,0.373]])

In [ ]:
RS_model.predict([[0.339,0.430]])-RA_model.predict([[0.307,0.373]])

In [ ]:
W_model.predict([[179.11250606]])

In [ ]:
## Applying KNN imputation Technique.

In [ ]:
from sklearn.impute import KNNImputer
Imp=KNNImputer(n_neighbors=6)
df_knn=pd.DataFrame(Imp.fit_transform(df1),columns=df1.columns)

In [ ]:
# Extracting our variables from the dataframe.
x = df_knn[['OOBP','OSLG']].values
y = df_knn[['RA']].values# Calling our model object.
RA_model = LinearRegression()# Fitting the model.
RA_model.fit(x,y)# Printing model intercept and coefficients.
print(RA_model.intercept_)
print(RA_model.coef_)

In [ ]:
# Extracting our variables from the dataframe.
x = df_knn[['OBP','SLG']].values
y = df_knn[['RS']].values# Calling our model object.
RS_model = LinearRegression()# Fitting the model.
RS_model.fit(x,y)# Printing model intercept and coefficients.
print(RS_model.intercept_)
print(RS_model.coef_)

In [ ]:
x = df_knn[['RD']].values
y = df_knn[['W']].values# Calling our model object.
W_model = LinearRegression()# Fitting the model.
W_model.fit(x,y)# Printing model intercept and coefficients.
print(W_model.intercept_)
print(W_model.coef_)

In [ ]:
# Prediction for runs scored.
RS_model.predict([[0.339,0.430]])

In [ ]:
RA_model.predict([[0.307,0.373]])

In [ ]:
RS_model.predict([[0.339,0.430]])-RA_model.predict([[0.307,0.373]])

In [ ]:
W_model.predict([[299.45173576]])

In [ ]:
## Applying Iterative Imputation
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
imp_mean = IterativeImputer(random_state=0)
df_it=pd.DataFrame(imp_mean.fit_transform(df1),columns=df1.columns)

In [ ]:
# Extracting our variables from the dataframe.
x = df_it[['OOBP','OSLG']].values
y = df_it[['RA']].values# Calling our model object.
RA_model = LinearRegression()# Fitting the model.
RA_model.fit(x,y)# Printing model intercept and coefficients.
print(RA_model.intercept_)
print(RA_model.coef_)

In [ ]:
# Extracting our variables from the dataframe.
x = df_it[['OBP','SLG']].values
y = df_it[['RS']].values# Calling our model object.
RS_model = LinearRegression()# Fitting the model.
RS_model.fit(x,y)# Printing model intercept and coefficients.
print(RS_model.intercept_)
print(RS_model.coef_)

In [ ]:
x = df_it[['RD']].values
y = df_it[['W']].values# Calling our model object.
W_model = LinearRegression()# Fitting the model.
W_model.fit(x,y)# Printing model intercept and coefficients.
print(W_model.intercept_)
print(W_model.coef_)

In [ ]:
# Prediction for runs scored.
RS_model.predict([[0.339,0.430]])

In [ ]:
RA_model.predict([[0.307,0.373]])

In [ ]:
W_model.predict(RS_model.predict([[0.339,0.430]])-RA_model.predict([[0.307,0.373]]))

In [ ]:
## Inference Iterative Imputer is providing much more accurate score than all other imputation methods.
## The actual score was 103, so Iterative Imputer provides very near to that score.
## Still the scope of non linear model is left out.